In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import time
import re
import pandas as pd
from urllib.request import urlopen
from urllib.parse import quote_plus

---
### 크롤링 - 인스타그램

In [ ]:
def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + word
    return url

def instagram_login():

    # 개인정보 보호 오류 창... 
    driver.find_element_by_css_selector('#details-button').click(); time.sleep(1)
    driver.find_element_by_css_selector('#proceed-link').click(); time.sleep(2)
    # 로그인
    driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').send_keys('아이디')
    driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').send_keys('비밀번호')
    driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button/div').click(); time.sleep(5)
    # 로그인 정보 저장 (나중에 하기)
    driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/div/div/button').click()
    time.sleep(3) 
    # 알림설정 (나중에 하기)
    driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div[3]/button[2]').click()
    time.sleep(3)

# 첫게시물 보기
def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)

# 다음게시글
def move_next(driver):
    right = driver.find_element_by_css_selector ('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)

# 내용 훔쳐오기
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ' '
        
    # 해시태그 > 정규표현식
    tags = re.findall(r'#[^\s#,\\]+', content) 
    # 작성일자
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    # 좋아요 수
    try:
        like = soup.select('a.zV_Nj')[0].text[4:-1]   
    except:
        like = 0
    # 위치정보
    try: 
        place = soup.select('div.M30cS')[0].text
    except:
        place = ''
    # 리스트로 저장
    data = [content, date, like, place, tags]
    return data

#### 텍스트 

In [ ]:
driver = webdriver.Chrome("chromedriver.exe")
driver.get('https://www.instargram.com')

instagram_login()

# 검색결과
driver.get(insta_searching('별의커비'))
time.sleep(4)

# 함수 차례대로 실행
select_first(driver)
results = [ ]
target = 33    # 가져올 게시글 수
for i in range(target):
    # 수집 오류 > 2초 대기
    try:
        data = get_content(driver)    # 게시글 정보
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)
    
driver.close()

In [ ]:
df2 = pd.DataFrame(results)
df2.columns = ['content','data','like','place','tags']
df2.to_excel('별의커비 해시태그.xlsx', index=False) 

pd.read_excel('별의커비 해시태그.xlsx')

,content,data,like,place,tags
0,⠀쉬는날엔 택배받는날모코모코 파우치😍,2020-04-13,115,NaN,[]
1,Ready to play🎮💖 #Kirby,2020-03-19,"2,262",NaN,['#Kirby']
2,カービィちゃんの体組成計🍎﻿﻿前から気になっていたけど、ヤマシロヤさんで半額😳とお安く...,2021-01-10,"1,077",NaN,"['#カービィ', '#星のカービィ', '#星之卡比', '#커비', '#별..."
3,🥺🍔🍟💖,2019-10-04,137,NaN,[]
4,"커비 컵, 와들디 컵 너무 귀여워💖 #Kirby",2020-07-24,"1,229",NaN,['#Kirby']
5,🍓🍊 #fanart #Kirby,2020-07-18,798,NaN,"['#fanart', '#Kirby']"
6,pink+🍊 #Kirby,2020-07-25,482,NaN,['#Kirby']
7,Kirby gathered together.ぎゅうぎゅうに盛り沢山。ーーーーーーーー...,2020-01-25,926,NaN,"['#Kirby', '#swordkirby', '#cleaningkirby', '#..."
8,D-1🧳✈️🏝️ #AnimalCrossing #fanart #Kirby,2020-03-18,"1,532",NaN,"['#AnimalCrossing', '#fanart', '#Kirby']"
9,결국은 이거였던 세연이의 큰그림.입학선물겸 생일선물로준비한 닌텐도스위치 라이트.맨날...,2021-02-25,6,NaN,"['#8살아들', '#생일', '#생일선물', '#입학선물', '#닌텐도스위치라이트..."


#### 이미지

In [ ]:
driver = webdriver.Chrome("chromedriver.exe")
driver.get('https://www.instargram.com')

instagram_login()

driver.get(insta_searching('별의커비'))
time.sleep(4)

html = driver.page_source
soup = BeautifulSoup(html)

insta = soup.select('.v1Nh3.kIKUG._bz0w')

n = 1
for i in insta:
    print('https://www.instagram.com'+ i.a['href'])
    imgUrl = i.select_one('.KL4Bh').img['src']
    with urlopen(imgUrl) as f:
        with open('./img/' + '별의커비' + str(n) + '.jpg', 'wb') as h:
            img = f.read()
            h.write(img)
    n += 1
    print(imgUrl)
    print()

driver.close()

https://www.instagram.com/p/B-546lYFEyr/
https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/sh0.08/e35/s640x640/93402320_732805950456744_4226925869123551691_n.jpg?_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=100&_nc_ohc=BpkwlywD0wMAX_GBCRG&tp=1&oh=200497aadbd2f6a964f2d6b3e213452b&oe=60607697

https://www.instagram.com/p/B96sx5YpOQB/
https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/sh0.08/e35/s640x640/82993872_1107027166328524_2975569197304642512_n.jpg?_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=104&_nc_ohc=b81FuxPnUlAAX-NFNxZ&tp=1&oh=7eb853e2eaa560e2bbd6b4c90d4d589d&oe=6060B4EF

https://www.instagram.com/p/CJ3ZS6dJl3i/
https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/sh0.08/e35/c187.0.706.706a/s640x640/136969411_223247452667856_7986089384224024910_n.jpg?_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=100&_nc_ohc=BSdjCk5onH8AX9KlEae&tp=1&oh=8aa1993aad74e682820c02acd3836714&oe=605F9682

https://www.instagram.com/p/B3MQWLSFtD9/
https://scontent-ssn1-1.cdninstagram.c

https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/sh0.08/e35/s640x640/152400381_113604820744106_4755813814080502875_n.jpg?_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=102&_nc_ohc=GUwRyXB6BRoAX_c-4rN&tp=1&oh=f28625bd541610aad8e66dba61737db3&oe=606230B6

https://www.instagram.com/p/CLoPy0ULBd7/
https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/sh0.08/e35/s640x640/152394486_434809114500645_3891162685663148391_n.jpg?_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=109&_nc_ohc=a5W4xupfHEsAX87XYOe&tp=1&oh=9ce30ea6428301dea97e8be8acbd8284&oe=60619142

https://www.instagram.com/p/CLn9ehPge4s/
https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/sh0.08/e35/s640x640/152397828_441365173581817_765830785497107688_n.jpg?_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=101&_nc_ohc=QEwai--6JcgAX-lT7IQ&tp=1&oh=e58b7193c871ed39de4d2bd240f8f453&oe=60621B88

https://www.instagram.com/p/CLn6CHADbJK/
https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/sh0.08/e35/c0.90.720.720a/s640x640/15236